<a href="https://colab.research.google.com/github/alihammadbaig/pdf_to_excel/blob/master/pdf_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive/data"

Mall_Customers.csv  pdf_to_excel.pdf


In [2]:
pip install tika

  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
Successfully built tika


In [0]:
import tika
import re

tika.initVM()
from tika import parser

parsed = parser.from_file('/content/gdrive/My Drive/data/pdf_to_excel.pdf')
# print(parsed["metadata"])
content = parsed["content"]
content = content.replace("Microsoft Word - Documents.docx", "").strip()
content = re.sub("\n*File [0-9]+:", "|||||", content)
print(content)

||||| 
File Name: Instructions.docx 
File Size: 2 MB 
Download Link: https://www.sendspace.com/file/r59by6 
Delete File 
Link: https://www.sendspace.com/delete/r59by6/57844a66dd1958f10f513c28bfe30c
1f 
 ||||| 
File Name: excel instructions.xlsx 
File Size: 995.5 KB 
Download Link: https://www.sendspace.com/file/lv6xjf 
Delete File 
Link: https://www.sendspace.com/delete/lv6xjf/7138a5de96bf4bf22d982b53dc56690
6 
 ||||| 
File Name: L290-AQ-MDR-13850_0_SECTN_88OF107_IPX Reviewed_AC-
OCR.pdf 
File Size: 173.3 MB 
Download Link: https://www.sendspace.com/file/mfj1z7 
Delete File 
Link: https://www.sendspace.com/delete/mfj1z7/1c77539be54f2415c7a24ce5427a27
81


In [0]:
section = content.split("|||||")
section = list(filter(None, section))
section = [re.sub(r'(File\s*\n*Name)', "File Name", line) for line in section]
section = [re.sub(r'(File\s*\n*Size)', "File Size", line) for line in section]
section = [re.sub(r'(Download\s*\n*Link)', "Download Link", line) for line in section]
section = [re.sub(r'(Delete\s*\n*File\s*\n*Link)', "Delete File Link", line) for line in section]

d = {}

i = 1
for line in section:
  d["File "+str(i)] = line
  i = i+1

for k, v in d.items():
  for line in v.splitlines():
    print(line)    
    print("----")
    print(line.split(" : "))

 
----
[' ']
File Name: Instructions.docx 
----
['File Name: Instructions.docx ']
File Size: 2 MB 
----
['File Size: 2 MB ']
Download Link: https://www.sendspace.com/file/r59by6 
----
['Download Link: https://www.sendspace.com/file/r59by6 ']
Delete File Link: https://www.sendspace.com/delete/r59by6/57844a66dd1958f10f513c28bfe30c
----
['Delete File Link: https://www.sendspace.com/delete/r59by6/57844a66dd1958f10f513c28bfe30c']
1f 
----
['1f ']
 
----
[' ']
 
----
[' ']
File Name: excel instructions.xlsx 
----
['File Name: excel instructions.xlsx ']
File Size: 995.5 KB 
----
['File Size: 995.5 KB ']
Download Link: https://www.sendspace.com/file/lv6xjf 
----
['Download Link: https://www.sendspace.com/file/lv6xjf ']
Delete File Link: https://www.sendspace.com/delete/lv6xjf/7138a5de96bf4bf22d982b53dc56690
----
['Delete File Link: https://www.sendspace.com/delete/lv6xjf/7138a5de96bf4bf22d982b53dc56690']
6 
----
['6 ']
 
----
[' ']
 
----
[' ']
File Name: L290-AQ-MDR-13850_0_SECTN_88OF107_IPX 

[' ']
['File Name: Instructions.docx ']
['File Size: 2 MB ']
['Download Link: https://www.sendspace.com/file/r59by6 ']
['Delete File ']
['Link: https://www.sendspace.com/delete/r59by6/57844a66dd1958f10f513c28bfe30c']
['1f ']
[' ']
[' ']
['File Name: excel instructions.xlsx ']
['File Size: 995.5 KB ']
['Download Link: https://www.sendspace.com/file/lv6xjf ']
['Delete File ']
['Link: https://www.sendspace.com/delete/lv6xjf/7138a5de96bf4bf22d982b53dc56690']
['6 ']
[' ']
[' ']
['File Name: L290-AQ-MDR-13850_0_SECTN_88OF107_IPX Reviewed_AC-']
['OCR.pdf ']
['File Size: 173.3 MB ']
['Download Link: https://www.sendspace.com/file/mfj1z7 ']
['Delete File ']
['Link: https://www.sendspace.com/delete/mfj1z7/1c77539be54f2415c7a24ce5427a27']
['81']


In [0]:
file = open("/content/gdrive/My Drive/data/pdf_to_excel.txt","w") 
file.write(content)
file.close()

FileNotFoundError: ignored

In [0]:
match = re.match(r"(^File [0-9]+):([^File [0-9]+].*)", content, re.I)
if match:
    print("match")
    items = match.groups()

items
# re.split("File d+", content)
# key, value = content.split("File ")
# my_dict = {key: value}

# dict(zip(*content.split(':')))
# dict(zip(*iter(content.split(':'))))

('File 1',)

In [3]:
import tika
import re

tika.initVM()
from tika import parser

parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/sample_pdf_for_excel.pdf')
# print(parsed["metadata"])
content = parsed["content"]






2019-06-04 10:45:56,399 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2019-06-04 10:45:57,235 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2019-06-04 10:45:57,492 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2019-06-04 10:46:02,500 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [5]:
c = content[:11000].strip()
page_num = 5479
start_of_page = "Page " + str(page_num) +" of 13699"
end_of_page = "Page " + str(page_num + 1) +" of 13699"
page = c[c.find(start_of_page): c.find(end_of_page)]

start_of_table = re.search("AVE\s+DATE", page).start()+8
end_of_table = re.search("FINAL\s+INSPECTION", page).start()-11
table = page[start_of_table:end_of_table]

# Data cleansing
table = table.replace("·","-")\
  .replace("•","")\
  .replace(" ' "," ")\
  .replace("l4", "14")

table = re.sub("OS\-([A-Z]{3})", r"05-\1", table,  flags=re.I)

print(table)
  



 
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106 112 209 224 238 247 210 226 04-Nov-14 05-Nov-14 

J20113 A2ED-702 CF241-1SBOV-02 7 102 118 111 118 113 112 202 219 233 203 219 215 04-Nov-14 05-Nov-14 -
3 S046836, A2EB-582 PL033-6S80-03 7 112 122 107 122 105 114 213 223 242 246 214 228 04-Nov-14 05-Nov-14 

4 5014458 A2EA-S92 PG015-1SCO-Q5 7 106 101 114 101 109 106 206 228 237 205 209 217 04-Nov-14 05-Nov-14 

5 S042449 A2El-SS2 AM005-1SA0-19 7 101 117 103 117 112 110 201 218 232 250 218 224 04-Nov-14 05-Nov-14 

6 S013917 A2EA-712 WS008-1SA0-05 7 111 121 112 121 104 114 210 222 241 202 213 218 04-Nov-14 05-Nov-14 

7 S024105 C4E8-420 PG008-1S80-Q2 7 105 100 108 125 114 110 205 227 236 245 217 226 04-Nov-14 05-Nov-14 

8 S020443 C7EA-410 PG159-1S80-02 7 100 116 115 116 108 111 200 217 231 204 208 212 04-Nov-14 05-Nov-14 

9 S01S848 A2E8-602 MR014-6SBO-o6 7 108 120 104 120 111 113 211 221 240 249 212 227 04-Nov-14 05-Nov-14 

10 S019248 A2EL-S52 AM142-1SA0.06 7 110 125 113 123

In [21]:
col_names = ["top_coat_date", "blast_primer_date"
             ,"dft1_ave", "dft1_5", "dft1_4", "dft1_3", "dft1_2", "dft1_1"
             ,"dft2_ave", "dft2_5", "dft2_4", "dft2_3", "dft2_2", "dft2_1"]
cols = {}

def get_col():
  global r
  r = r.strip()
  val = r[r.rfind(" ") : len(r)]
  r = r[0 : r.rfind(" ")]
  print(r)
  return val

for row in [x for x in table.splitlines() if x]:
  row = row.strip()
  row = re.sub("-\s*$", "", row)

  global r
  r = row
  
  for col_name in col_names:
    col_val = get_col()
    cols[col_name] = col_val
  
#   for k,v in cols.items():
#     print("{0}:{1}".format(k,v))
    
#   last_space = row.rfind(" ")
#   row_end = len(row)
#   top_coat_date = row[last_space : row_end]
#   print("last space:%i" % last_space)
#   print("row end:%i" % row_end)
  
















1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106 112 209 224 238 247 210 226 04-Nov-14
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106 112 209 224 238 247 210 226
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106 112 209 224 238 247 210
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106 112 209 224 238 247
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106 112 209 224 238
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106 112 209 224
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106 112 209
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106 112
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124 106
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102 124
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123 102
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107 123
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7 107
1 S031109 A2EL-SS2 AM146-1SAO-HK-01 7
J20113 A2ED-702 CF241-1SBOV-02 7 102 118 111 118 113 112 202 219 233 203 219 215 04-